In [0]:
CREATE LIVE TABLE diagnostic_mapping
COMMENT "Bronze table for diagnostic map"
TBLPROPERTIES ("quality"="bronze")
AS
SELECT * FROM incremental_load.default.raw_diagnosis_map

In [0]:
CREATE OR REFRESH STREAMING TABLE daily_patients
COMMENT "Bronze table for daily patients"
TBLPROPERTIES ("quality"="bronze")
AS
SELECT * FROM STREAM(incremental_load.default.raw_daily_patients)

In [0]:
CREATE OR REFRESH STREAMING TABLE processed_patients_data
COMMENT "Silver table for processed patients data"
TBLPROPERTIES ("quality"="silver")
AS
SELECT
  p.patient_id,
  p.name,
  p.age,
  p.gender,
  p.address,
  p.contact_number,
  p.admission_date,
  m.diagnosis_description
FROM STREAM(live.daily_patients) p
LEFT JOIN live.diagnostic_mapping m
ON p.diagnosis_code = m.diagnosis_code;

In [0]:
CREATE LIVE TABLE patient_statistics_by_diagnosis
COMMENT "Gold table for patient statistics by diagnosis"
TBLPROPERTIES ("quality"="gold")
AS
SELECT
  diagnosis_description,
  COUNT(patient_id) as patient_count,
  AVG(age) as avg_age,
  COUNT(DISTINCT gender) as unique_gender_count,
  MIN(age) as min_age,
  MAX(age) as max_age
FROM live.processed_patients_data
GROUP BY diagnosis_description;

In [0]:
CREATE LIVE TABLE patient_statistic_by_gender
COMMENT "Gold table for patient statistics by gender"
TBLPROPERTIES ("quality"="gold")
AS
SELECT
  gender,
  COUNT(patient_id) as patient_count,
  AVG(age) as avg_age,
  COUNT(DISTINCT diagnosis_description) as unique_diagnosis_count,
  MIN(age) as min_age,
  MAX(age) as max_age
FROM live.processed_patients_data
GROUP BY gender;